In [3]:
# Import Dependencies
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
csv_path_flow_2023 = Path("user_flow_fy2023.csv")
csv_path_dist_flow = Path("6. district_flow_estimate.csv")
csv_path_verif_flow = Path("7. verification_flow.csv")
dist_flow = pd.read_csv(csv_path_dist_flow, encoding="UTF-8")
flow_2023 = pd.read_csv(csv_path_flow_2023, encoding="UTF-8")                         
VerificationFlow_2023 = pd.read_csv(csv_path_verif_flow, encoding="UTF-8")
VerificationFlow_2023.round(2)

,station,report_year,annual_flow
0,Main,2023,47.98
1,O Street,2023,4.16
2,Eastside,2023,5.40
3,Poplar Point,2023,7.76
4,Potomac,2023,111.65
5,Rock Creek,2023,3.22
6,Upper Anacostia,2023,1.20
7,Earl Place,2023,0.01


In [4]:
distric_flow_est =  pd.read_csv(csv_path_dist_flow, encoding="UTF-8")
distric_flow_est

,user,flow_route,flow_route_name,sewersheds,pop,emp,visitor,area_acres,infilt_InchMile,Infil_mgdinchMile,...,pro_infiltration_rate,storm_mgd,sanitary_mgd,high_user_mgd,ground_water_pumpage,total_annual_flow,billing_dc_flow,impervious__acres,pervious_acres,pipe_surface_area_acres
0,dc,1,little falls,MD-LF,330,874,209,43,0.00,0.0000,...,NaN,NaN,0.09,NaN,0.02,NaN,NaN,NaN,NaN,0.67
1,dc,1,little falls,LF Boundary (BN-68),10084,8478,0,870,781.85,0.0026,...,NaN,NaN,1.42,0.16,0.04,NaN,NaN,NaN,NaN,10.86
2,dc,1,little falls,UPI-NM1 (BN-68),259,431,0,48,13.03,0.0026,...,NaN,NaN,0.04,0.49,NaN,NaN,NaN,NaN,NaN,0.36
3,dc,1,little falls,UPI-NM2 (BN-69),2241,1693,0,202,151.42,0.0026,...,NaN,NaN,0.31,NaN,NaN,NaN,NaN,NaN,NaN,1.91
4,dc,1,little falls,UPI-NM3 (M-4317),585,440,0,107,108.90,0.0026,...,NaN,NaN,0.08,0.24,NaN,NaN,NaN,NaN,NaN,1.95
5,dc,2,upper potomac interceptor,UPI-7,2255,785,0,281,203.23,0.0000,...,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,3.45
6,dc,2,upper potomac interceptor,UPI-NM4,2567,695,0,360,177.35,0.0000,...,NaN,NaN,0.31,NaN,NaN,NaN,NaN,NaN,NaN,3.77
7,dc,2,upper potomac interceptor,UPI-1,18890,9136,0,1182,927.36,0.0000,...,NaN,NaN,2.41,0.60,0.04,NaN,NaN,NaN,NaN,10.08
8,dc,2,upper potomac interceptor,CS0 029 (College Pond),8793,6052,154,321,376.32,0.0000,...,NaN,0.46,1.21,0.41,0.08,NaN,NaN,132.0,189.0,0.71
9,dc,2,upper potomac interceptor,CSO 057/058,5005,1998,1441,177,298.08,0.0000,...,NaN,0.27,0.78,0.20,0.04,NaN,NaN,86.0,91.0,5.04


# Inputs

In [20]:
rainfall = 42
#1 ac-ft/min = 
gal_per_min = 325851.428571
Drainage_coeff_for_imperv_area = 0.9
#ratio of runoff to rainfall (Design Manual, Vol. 3, p. 3-D1-7), in./in.
Drainage_Coeff_for_perv_Area = 0.3
Wastewater_per_capita =110
# mgd (From Section 3.5.2 Unit Wastwater Flow Factor)
Theoretical_Qlf = 16.6
Factor_for_High_Water_Use= Theoretical_Qlf/19.05
# Infiltration (Table 6-1)= ((rainfall-40.84)*((50.32-40.84)/(52-64)))+52
Infiltration = (((rainfall-40.84)*((50.32-40.84)/(52-64)))+52)
Factor_infil= Infiltration/distric_flow_est["infil_MGD"].sum()
#  gallons/day from Tech Memo 18, p. 3-14
# Rainfall 34.90 inches	Comparing to Table 6-1 'Projected District Wastewater Quantities'
# Rainfall ft = 2.908333 ac-ft/min 325,851 gal/min

In [5]:
print("Rain Fall 2023                 =",rainfall)
print("Infiltration based on rainfall = ",format(Infiltration, '.2f'))
print("Infiltration Factor            = ",format(Factor_infil, '.2f') )
print("Drainage_coeff_for_imperv_area =",Drainage_coeff_for_imperv_area)
print("Drainage_Coeff_for_perv_Area   = ", Drainage_Coeff_for_perv_Area)
print("Wastewater_per_capita          = ",Wastewater_per_capita)
print("Theoretical_Qlf                = ",Theoretical_Qlf)
print("Factor for High Water Use      = ", format(Factor_for_High_Water_Use, '.2f'))

NameError: name 'rainfall' is not defined

Calculate District Sewershed Flows incl Sanitary, Storm, High Water Users and Goundwater, finally Total district flow

In [22]:
distric_flow_est['pro_infiltration_rate'] = distric_flow_est["infil_MGD"]*Factor_infil
distric_flow_est['storm_mgd'] = (distric_flow_est["impervious__acres"]*Drainage_coeff_for_imperv_area*(rainfall/12)*gal_per_min
                                + 
                                distric_flow_est["pervious_acres"]*Drainage_Coeff_for_perv_Area* (rainfall/12)*gal_per_min)/365.242199/1000000
distric_flow_est['storm_mgd'] =distric_flow_est['storm_mgd'] .fillna(0)

distric_flow_est['sanitary_mgd'] = ((distric_flow_est["pop"]+ distric_flow_est["emp"]/3+ 
                                    distric_flow_est["visitor"])*Wastewater_per_capita )/1000000
distric_flow_est['high_user_mgd'] = distric_flow_est['high_user_mgd'].fillna(0)
distric_flow_est['ground_water_pumpage'] = distric_flow_est['ground_water_pumpage'].fillna(0)
# distric_flow_est['DataFrame Column'] = distric_flow_est['DataFrame Column'].fillna(0)
                 
distric_flow_est['total_annual_flow'] = (distric_flow_est["storm_mgd"]+ distric_flow_est["sanitary_mgd"] +\
                                    distric_flow_est["high_user_mgd"]+distric_flow_est["ground_water_pumpage"] +\
                                    distric_flow_est["pro_infiltration_rate"]).round(2)

district_flow = distric_flow_est[["flow_route","sewersheds","storm_mgd","sanitary_mgd","pro_infiltration_rate",
                  "high_user_mgd","ground_water_pumpage","total_annual_flow"]]

district_flow = district_flow[["flow_route","sewersheds","total_annual_flow"]]
DC_est_annualflow = district_flow["total_annual_flow"].sum()
print("Estimated District Flow = ",DC_est_annualflow,"mgd")
district_flow.head()

Estimated District Flow =  190.43 mgd


,flow_route,sewersheds,total_annual_flow
0,1,MD-LF,0.11
1,1,LF Boundary (BN-68),2.60
2,1,UPI-NM1 (BN-68),0.55
3,1,UPI-NM2 (BN-69),0.50
4,1,UPI-NM3 (M-4317),0.46


In [8]:
print("Storm, mgd = ",distric_flow_est['storm_mgd'].sum().round(2))
print("Sanitary, mgd = ",distric_flow_est['sanitary_mgd'].sum().round(2))
print("Infiltration,mgd = ",distric_flow_est['pro_infiltration_rate'].sum().round(2))
print("TotalAnnualFlow, mgd = ",distric_flow_est['total_annual_flow'].sum().round(2))

Storm, mgd =  23.14
Sanitary, mgd =  89.05
Infiltration,mgd =  51.08
TotalAnnualFlow, mgd =  190.43


# User Flow Reported Fiscal Year 2023

Flow Share calculation  Inside the District Boundary

# WSSC Billing Flow Report for Fiscal Year 2023

In [6]:
flow_2023_wssc = flow_2023[flow_2023["user_name"]=='wssc']
flow_2023_wssc_flow = flow_2023_wssc.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("WSSC Billing Total Flow = ",flow_2023_wssc_flow.sum().map("{:,.2f}".format))
flow_2023_wssc_flow.round(2)

WSSC Billing Total Flow =  monthly_flow    97.16
dtype: object


,monthly_flow
input_name,
BARNABY,0.76
BELT,0.16
CABIN_JOHN,5.68
DC-WSSC - ANACOSTIA,52.12
EAST BEACH DRIVE,0.70
EASTBEACH_DRIVE,0.70
LITTLE_FALLS,3.00
MD. UPI,6.13
MUDDY,6.37


# Fairfax Billing Flow Report for Fiscal Year 2023

In [8]:
flow_2023_fairfax = flow_2023[flow_2023["user_name"]=='fairfax']
flow_2023_fairfax_flow = flow_2023_fairfax.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("Fairfax Billing Total Flow = ",flow_2023_fairfax_flow.sum().map("{:,.2f}".format))
flow_2023_fairfax_flow.round(2)

Fairfax Billing Total Flow =  monthly_flow    16.83
dtype: object


,monthly_flow
input_name,
AT & T - B3046 - P.I.,6.26
CHAIN BRIDGE - G4102 - DOWN,1.88
ROCK HILL ROAD - A3003 - P.I.,0.57
SUGARLAND RUN - B2006 - P.I.,3.51
Sully_Rd_1,2.69
Sully_Rd_2,1.88
TREESIDE - B5706 - P.I.,0.01
UNMETERED,0.02


# Loudoun Billing Flow Report for Fiscal Year 2023

In [9]:
flow_2023_Loudoun = flow_2023[flow_2023["user_name"]=='loudoun']
flow_2023_Loudoun_flow = flow_2023_Loudoun.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("Loudoun Billing Total Flow = ",flow_2023_Loudoun_flow.sum().map("{:,.2f}".format),"mgd")
flow_2023_Loudoun_flow.round(2)

Loudoun Billing Total Flow =  monthly_flow    6.59
dtype: object mgd


,monthly_flow
input_name,
BEAUMEADE #1,0.22
BEAUMEADE 2 SM,0.02
BOISE CASCADE,0.37
BRIDGES 625/604 - S8,0.45
BROAD RUN,0.31
CABIN BRANCH,1.44
CASCADES NORTH - P.L. NORTH,0.23
CASCADES WESTERN,0.29
COUNTRYSIDE #1,0.23


# Vienna Billing Flow Report for Fiscal Year 2023

In [26]:
flow_2023_Vienna = flow_2023[flow_2023["user_name"]=='vienna']
flow_2023_Vienna_flow = flow_2023_Vienna.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("Town of Vienna Billing Total Flow = ",flow_2023_Vienna_flow.sum().map("{:,.2f}".format))
flow_2023_Vienna_flow.round(2)

Town of Vienna Billing Total Flow =  monthly_flow    1.06
dtype: object


,monthly_flow
input_name,
CREEK CROSSING,0.43
NORTHSIDE,0.53
UNMETERED - VIENNA,0.11


# Dulles International Airport Billing Flow Report for Fiscal Year 2023

In [27]:
flow_2023_Dulles = flow_2023[flow_2023["user_name"]=='dulles']
flow_2023_Dulles_flow = flow_2023_Dulles.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("Dulles Billing Total Flow = ",flow_2023_Dulles_flow.sum().map("{:,.2f}".format))
flow_2023_Dulles_flow.round(2)

Dulles Billing Total Flow =  monthly_flow    0.81
dtype: object


,monthly_flow
input_name,
DULLES - S -110,0.77
DULLES - S -14,0.04


In [15]:
#Prorate District Flow
# for i in range(90):
#     percent_diverted_to_MainPS=10
#     Main_Verif_Flow = VerificationFlow_2023.loc[[0,'annual_flow']]
#     Suburban_flow_into_MainPS = [(flow_2023_wssc_flow["ROCKCREEK_VENTURI_36"]+flow_2023_wssc_flow["ROCKCREEK_VENTURI_36"])].sum()["monthly_flow"] *percent_diverted_to_MainPS
#     Main_PS_DC_Calc = Main_Verification_Flow - Suburban_flow_into_MainPS
#     Adjust-Coeff_main = Main_PS_DC_Calc/Main_PS_DC_Est
#     for j in range (len(district_flow ))
#     Main_PS_DC_Est = Main_PS_DC_Est*Adjust-Coeff_main

#     Potomac_Verification_Flow = VerificationFlow_2023.loc[4,'annual_flow']
#     Suburban_flow_into_PotomacPS = WSSC_RCMI*(1-percent_diverted_to_MainPS)+PI_Users_Flow
#     PotomacPS_DC_Calc = Potomac_Verification_Flow- Suburban_flow_into_PotomacPS
#     Adjust-Coeff_Potomac = PotomacPS_DC_Calc/Potomac_PS_DC_Est
#     Potomac_PS_DC_Est = Potomac_PS_DC_Est*Adjust-Coeff_Potomac

# if Adjust-Coeff_Potomac == Adjust-Coeff_main
#     print("Iteration Completed")
#     else
    

In [ ]:
#Prorate District Flow
For i = 0.1

Split_at_str35 = i
PS_DC = RCPS + RTE3 + RTE4_1of2
MS_DC = RTE4_2of2 + DC1 + DC1 

Adjcoeff_PS = PS/PS_DC_Est
Adjceff_MS = MS_Ver/MS_DC_Est

if Adjcoeff_PS = Adjceff_MS then



Route 1 to 11 Pipes Inside

Route 12 Potomac Interceptor

In [ ]:
# Route 12A

# Route 12B

# Route 12C

# Route 12E

# Route 12F

